In [1]:
import openai
import json
import pandas as pd
from pprint import pprint

In [2]:
# Set up the ChatGPT API credentials
openai.api_key = json.load(open('key.json', 'r'))['key']

data_path = "chatgpt_training_data_v1.0.tsv"
df = pd.read_csv(data_path, sep='\t')
df.columns = ['question', 'project_dashboard', 'result', 'query', 'concat']
examples = "\n".join(df.apply(lambda x: f"{x['question']}: {x['result']}", axis=1))

In [76]:
def ask_gpt(examples=examples, question=None, prepend_question=False, prelude=None):
    prompt = examples if question is None else f"{examples}\n\n{question}:"
    if prelude is not None:
        prompt = prelude + '\n\n' + prompt
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.7
    )
    answer = response.choices[0].text.strip()
    if prepend_question:
        return f"Q: {question}<br>A: {answer}<br><br>"
    else:
        return answer

In [82]:
from functools import reduce
from collections import defaultdict

keys = df['result'].apply(json.loads).apply(set)
values = df['result'].apply(json.loads).apply(lambda x: set(x.values()))
key_values = df['result'].apply(json.loads).to_list()
all_keys = reduce(lambda x, y: x | y, keys)
all_values = reduce(lambda x, y: x | y, values)
all_key_values = defaultdict(set)
for kvs in key_values:
    for k, v in kvs.items():
        if k.endswith('_1') or k.endswith('_2'):
            k = k[:-1] + 'i'
        all_key_values[k].add(v)
all_key_values

json_keys_str = "\n".join([f"K{i+1}. '{k}'" for i, k in enumerate(sorted(all_keys))])
# print(f'Allowed JSON keys are the following (from K1 to K{len(all_keys)}):')

# print(json_keys_str)
print('')
prelude = f'Allowed JSON keys are K1--K{len(all_keys)}, and range of values for key Ki are Vi.1--Vi.ni, where ni is the number of allowed values of key Ki. If you don\'t find any appropriate key or value, return the whole answer as NA (with reason included):'
json_kvs_str = '\n'.join([f"K{i+1}. '{k}'\n\t{', '.join(['V' + str(i+1) + '.' + str(j+1) + '. ' + v for j, v in enumerate(sorted(all_key_values[k[:-1] + 'i' if k.endswith('_1') or k.endswith('_2') else k]))])}" for i, k in enumerate(sorted(all_keys))])

prelude += '\n'+json_kvs_str
print(prelude)


Allowed JSON keys are K1--K10, and range of values for key Ki are Vi.1--Vi.ni, where ni is the number of allowed values of key Ki. If you don't find any appropriate key or value, return the whole answer as NA (with reason included):
K1. 'end_time'
	V1.1. 31-jan-2023, V1.2. default
K2. 'query_breakdown_1'
	V2.1. $channel, V2.2. $initial_channel, V2.3. none
K3. 'query_breakdown_2'
	V3.1. $channel, V3.2. $initial_channel, V3.3. none
K4. 'query_entity_1'
	V4.1. $form_submitted, V4.2. average_page_views_per_session, V4.3. form_submission, V4.4. new_users, V4.5. repeat_users, V4.6. total_sessions, V4.7. unique_users, V4.8. website_session
K5. 'query_entity_2'
	V5.1. $form_submitted, V5.2. average_page_views_per_session, V5.3. form_submission, V5.4. new_users, V5.5. repeat_users, V5.6. total_sessions, V5.7. unique_users, V5.8. website_session
K6. 'query_filter_1'
	V6.1. is_first_session equals true, V6.2. none
K7. 'query_filter_2'
	V7.1. is_first_session equals true, V7.2. none
K8. 'query_ty

In [85]:
question = input('Q: ')

if len(question) < 5:
    answer = 'Your question should be at least 5 characters long.'
else:
    try:
        answer = ask_gpt(question=question, prelude=prelude)
    except openai.error.AuthenticationError:
        openai.api_key = input('API Key Error. Enter correct key: ')
        answer = ask_gpt(question=question, prelude=prelude)
print('Q: ', end='')
print(question)
print('A: ', end='')
try:
    pprint(json.loads(answer))
except:
    print(answer)

Q: how many sessions last to last month?
A: NA (time range allowed values do not include 'last to last month' value)


In [73]:
pprint(df['question'].to_list())

['Total Sessions',
 'Total number of Sessions in the last month',
 'Total number of Sessions in the last week',
 'How many new users we had this week?',
 'How many users this month visited our website this month?',
 'How many repeat users did we have in last quarter?',
 "What's the breakdown of website sessions by channel this quarter",
 'Sessions By Channel',
 "What's the average page views and time spent per session in the month of Jan",
 'First Session that led to Form Button Clicks',
 "What's the conversion rate from first session to a Form Button Click",
 'Which Channels are leading to Form Button Clicks today']
